## Programación Cuadrática

Este cuaderno muestra varias formas de resolver el problema
de optimización de carteras modelado con programación cuadrática.

In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import cvxpy as cp

### Datos

In [ ]:
import pickle

In [ ]:
with open('../data/stock_data.pkl', 'rb') as handle:
    stock_data = pickle.load(handle)

In [ ]:
close_dict = {tk: df.close for tk,df in stock_data.items()}
stock_close = pd.DataFrame(close_dict)

In [ ]:
data_close = stock_close.loc['2019-01-02':'2019-12-31'].dropna(axis=1)

Para simplificar el ejercicio trabajaremos con un universo reducido
de 10 activos del IBEX.

In [ ]:
data_close = data_close[['ACS','TEF','ITX','GRF','AMS','ENG','MAP','REP','AENA','VIS']]
data_close.head()

___

Calculamos la media de los retornos logaritmicos y la matriz de covarianzas de los
retornos

In [ ]:
returns = np.log(data_close).diff().dropna()
cov_returns = returns.cov()
mean_returns = returns.mean()

In [ ]:
mean_returns

In [ ]:
cov_returns

___

#### Espacio de posibles carteras
Simulamos primero un conjunto aleatorio de pesos para visualizar el espacio
de posibles carteras 

In [ ]:
n = mean_returns.shape[0]
list_ret_p = []
list_std_p = []
list_w = []
for _ in range(10000):
    w = np.random.rand(n)           # pesos aleatorios   
    w = w / np.sum(w)               # Escalamos para que sumen 1
    ret_p = mean_returns.dot(w)
    std_p = np.sqrt(w @ cov_returns.values @ w)  
    list_ret_p.append(ret_p)
    list_std_p.append(std_p)
    list_w.append(w)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 7))
ax.plot(list_std_p, list_ret_p, '.')

___

### Cartera de Varianza Mínima 
Primero haremos un modelo para construir la cartera
de varianza mínima

In [ ]:
Sigma = cov_returns.values
mu = mean_returns.values

In [ ]:
mu

In [ ]:
w = cp.Variable(n)

port_risk = cp.quad_form(w, Sigma)
port_ret = mu @ w
constraints = [
    cp.sum(w) == 1, 
    w >= 0
]
prob = cp.Problem(cp.Minimize(port_risk), 
                  constraints) 
prob.solve()

In [ ]:
cartera = pd.Series(w.value, index=returns.columns).round(3)
cartera.plot.pie()

In [ ]:
cartera

In [ ]:
port_risk.value, port_ret.value

Vemos donde se situa el portfolio de varianza mínima en el 
espacio de carteras

In [ ]:
port_std = np.sqrt(port_risk.value)
port_return =  port_ret

In [ ]:
fig, ax = plt.subplots(figsize=(10, 7))
ax.plot(list_std_p, list_ret_p, '.')
ax.plot(port_std, port_return.value, '*', markersize=15)

___

### Calculo de la Frontera Eficiente

In [ ]:
def efficient_frontier(returns, n_samples=50, gamma_low=-1, gamma_high=10):
    """
    construye un conjunto de problemas de programación cuádrática
    para inferir la frontera eficiente de Markovitz. 
    En cada problema el parámetro gamma se cambia para aumentar
    la penalización del riesgo en la función de maximización.
    """
    sigma = returns.cov().values
    mu = np.mean(returns, axis=0).values  
    n = sigma.shape[0]        
    w = cp.Variable(n)
    gamma = cp.Parameter(nonneg=True)
    ret = mu.T @ w
    risk = cp.quad_form(w, sigma)
    
    prob = cp.Problem(cp.Maximize(ret - gamma*risk), 
                      [cp.sum(w) == 1,  w >= 0]) 
    # Equivalente 
    #prob = cp.Problem(cp.Minimize(risk - gamma*ret), 
    #                  [cp.sum(w) == 1,  w >= 0])   
    risk_data = np.zeros(n_samples)
    ret_data = np.zeros(n_samples)
    gamma_vals = np.logspace(gamma_low, gamma_high, num=n_samples)
    
    portfolio_weights = []    
    for i in range(n_samples):
        gamma.value = gamma_vals[i]
        prob.solve()
        risk_data[i] = np.sqrt(risk.value)
        ret_data[i] = ret.value
        portfolio_weights.append(w.value)   
    return ret_data, risk_data, gamma_vals, portfolio_weights



In [ ]:
ret_data, risk_data, gamma_vals, portfolio_weights = efficient_frontier(returns)

In [ ]:
#portfolio_weights

### Cartera Óptima
Para determinar la **cartera con mejor ratio sharpe**, sacamos todos los
ratios Sharpe y elegimos donde ocurre el máximo

In [ ]:
sharpes = ret_data/risk_data 
idx = np.argmax(sharpes)
optimal_ret, optimal_risk = ret_data[idx], risk_data[idx]
optimal_portfolio = pd.Series(portfolio_weights[idx],
                              index=returns.columns).round(3)

In [ ]:
sharpes

Dibujamos la frontera eficiente y donde ocurre la cartera optima

In [ ]:
fig, ax = plt.subplots(figsize=(10, 7))
plt.plot(list_std_p, list_ret_p, '.')
ax.plot(risk_data, ret_data, 'y')
ax.plot(optimal_risk, optimal_ret, '*', markersize=15)
_ = ax.set_xlabel('std')
_ = ax.set_ylabel('mean')


In [ ]:
optimal_portfolio.plot.pie()

In [ ]:
optimal_portfolio

___

### Ejercicios Propuestos

1. Para evitar grandes posiciones grandes que puedan surgir en la cartera óptima limitaremos 
el tamaño máximo que se puede asignar a cada peso.  Para este ejercicio proponemos:
    - modificar la función de la frontera eficiente para que acepte las restricciones de peso máximo y peso mínimo por posición 
    - añadir al gráfico original las fronteras eficientes correspondientes a carteras que tienen posiciones máximas de 20 y 15%
2. Para centrarnos en empresas de más capitalización hemos decidido que las empresas que tienen menos de 10 mil millones de capitalización no deben sumar más del 25% de la cartera. En nuestro universo de activos esto corresponde a ACS, MAP, ENG y VIS.
